In [15]:
!pip install librosa
!pip install tqdm

You should consider upgrading via the '/home/jack/anaconda3/envs/exp38/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 72 kB 139 kB/s eta 0:00:01
You should consider upgrading via the '/home/jack/anaconda3/envs/exp38/bin/python -m pip install --upgrade pip' command.


In [16]:
import librosa
from librosa import display
import os
import pandas as pd
from glob import glob
from tqdm import tqdm
import time
import numpy as np

In [7]:
def extract_mfcc(file):
    #Load librosa array, obtain mfcss, store the file and the mcss information in a new array
    X, sample_rate = librosa.load(file, res_type='kaiser_fast')
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0) 
    # The instruction below converts the labels (from 1 to 8) to a series from 0 to 7
    # This is because our predictor needs to start from 0 otherwise it will try to predict also 0.
  # If the file is not valid, skip it
    
    return mfccs

In [86]:
class_map = {'angry':0, 'disgust':1, 'fearful':2,'happy':3,'neutral':4,'sad':5,'surprised':6}
def data_extract(data):
    data_path = data+'/*/*.wav' 
    img_paths = glob(data_path)
    data_count = len(glob(data_path))
    print(data_count)

    X = np.zeros((data_count, 40))
    y = np.zeros((data_count, ))
    for i, path in tqdm(sorted(enumerate(img_paths))):

        X[i] = extract_mfcc(path)

        cls = path.split('/')[-2]

        y[i] = class_map[cls]
    return X,y

# Train data Extract

In [118]:
X,y =data_extract('train') 

  0%|          | 2/1963 [00:00<01:38, 19.91it/s]

1963


100%|██████████| 1963/1963 [02:14<00:00, 14.65it/s]


In [119]:
import joblib
import keras
X_name = 'X_train.joblib'
y_name = 'y_train.joblib'

y_onehot = keras.utils.to_categorical(y, num_classes=7)
print(y.shape)
savedX = joblib.dump(X,X_name)
savedy = joblib.dump(y_onehot, y_name)

(1963,)


In [147]:
X_train = joblib.load('X_train.joblib')
y_train = joblib.load('y_train.joblib')



x_traincnn = np.expand_dims(X_train, axis=2)
print(x_traincnn.shape,y_train.shape)
print(y_train[0])
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Flatten, Dropout, Activation
from tensorflow.python.framework import tensor_util

from tensorflow.keras.layers import Conv1D, MaxPooling1D,LSTM,LeakyReLU,Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

model = Sequential()

model.add(Conv1D(16, 5,padding='same',
                 input_shape=(40,1)))
model.add(LeakyReLU())
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(2)))
model.add(Conv1D(32, 5,padding='same'))
model.add(LeakyReLU())
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(2)))
model.add(Conv1D(64, 5,padding='same',))
model.add(LeakyReLU())
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(2)))
model.add(Conv1D(128,5,padding='same',))
model.add(LeakyReLU())

model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(2)))
model.add(Conv1D(256, 5,padding='same',))
model.add(LeakyReLU())
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(2)))

model.add(Flatten())
model.add(Dense(128))

model.add(LeakyReLU())
model.add(Dense(7))
model.add(Activation('softmax'))

model.summary()

(1963, 40, 1) (1963, 7)
[0. 1. 0. 0. 0. 0. 0.]
Model: "sequential_39"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_168 (Conv1D)          (None, 40, 16)            96        
_________________________________________________________________
leaky_re_lu_184 (LeakyReLU)  (None, 40, 16)            0         
_________________________________________________________________
dropout_167 (Dropout)        (None, 40, 16)            0         
_________________________________________________________________
max_pooling1d_69 (MaxPooling (None, 20, 16)            0         
_________________________________________________________________
conv1d_169 (Conv1D)          (None, 20, 32)            2592      
_________________________________________________________________
leaky_re_lu_185 (LeakyReLU)  (None, 20, 32)            0         
_________________________________________________________________
dropou

In [149]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print(x_traincnn.shape, y_train.shape)
cnnhistory=model.fit(x_traincnn, y_train, batch_size=128, epochs=1000)

(1963, 40, 1) (1963, 7)
Epoch 1/1000


InvalidArgumentError:  logits and labels must have the same first dimension, got logits shape [128,7] and labels shape [896]
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at <ipython-input-149-478907783404>:5) ]] [Op:__inference_train_function_1080262]

Function call stack:
train_function


# Test data Extract

In [91]:
model.load_weights('ASR1.h5')
import joblib
import keras

X_name = 'X_test.joblib'
y_name = 'y_test.joblib'
'''
x_testcnn,y_test =data_extract('test') 
y_onehot = keras.utils.to_categorical(y_test, num_classes=7)
joblib.dump(x_testcnn,X_name)
joblib.dump(y_onehot, y_name)
'''

  1%|          | 3/489 [00:00<00:23, 20.97it/s]

489


100%|██████████| 489/489 [00:40<00:00, 12.10it/s]


# Show Result

In [92]:
x_testcnn = joblib.load('X_test.joblib')
y_test = joblib.load('y_test.joblib')
prediction_name = 'speech_prediction'
x_testcnn = np.expand_dims(x_testcnn, axis=2)
from sklearn.metrics import classification_report,confusion_matrix
y_pred = np.argmax(model.predict(x_testcnn), axis=-1)
y_test = np.argmax(y_test, axis=-1)
savedprediction = joblib.dump(y_pred, prediction_name)
target_names = ['angry', 'disgust', 'fearful','happy','neautral','sad','surprised']
print(classification_report(y_test, y_pred, target_names=target_names))
print(confusion_matrix(y_test,y_pred))